In [7]:
!pip install resemblyzer

  Using cached Resemblyzer-0.1.1.dev0-py3-none-any.whl (15.7 MB)
  Using cached webrtcvad-2.0.10.tar.gz (66 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached typing-3.7.4.3-py3-none-any.whl

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [9 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.7
  copying webrtcvad.py -> build\lib.win-amd64-3.7
  running build_ext
  building '_webrtcvad' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for webrtcvad
  error: subprocess-exited-with-error
  
  Running setup.py install for webrtcvad did not run successfully.
  exit code: 1
  


  Running setup.py clean for webrtcvad
Failed to build webrtcvad
  Running setup.py install for webrtcvad: started
  Running setup.py install for webrtcvad: finished with status 'error'



  [11 lines of output]
  running install
  c:\users\zlatt\anaconda3\lib\site-packages\setuptools\command\install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
    setuptools.SetuptoolsDeprecationWarning,
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.7
  copying webrtcvad.py -> build\lib.win-amd64-3.7
  running build_ext
  building '_webrtcvad' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: legacy-install-failure

Encountered error while trying to install package.

webrtcvad

note: This is an issue with the package mentioned above, not pip.
hint: See above for output from the failure.


In [2]:
!pip install umap

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3564 sha256=423d7ef003ebdab0e48160fdc47fc4f6a2d66a4c2841bec87dcdeeff4a320a25
  Stored in directory: c:\users\zlatt\appdata\local\pip\cache\wheels\65\55\85\945cfb3d67373767e4dc3e9629300a926edde52633df4f0efe
Successfully built umap


You should consider upgrading via the 'c:\users\zlatt\anaconda3\python.exe -m pip install --upgrade pip' command.


In [5]:
from resemblyzer import preprocess_wav, VoiceEncoder
from itertools import groupby
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Демонстрации упорядочены таким образом, чтобы пояснения в комментариях были последовательными. 
# Если вы заботитесь только о выполнении кода, то вам не обязательно следовать этому порядку.
# ДЕМОНСТРАЦИЯ 01: мы покажем, как сравнивать сегменты речи (= высказывания) между собой, 
# чтобы получить показатель того, насколько похоже звучат их голоса. 
# Мы ожидаем, что высказывания одного и того же говорящего будут иметь высокое сходство, 
# а высказывания разных говорящих - более низкое.

# Нейронная сеть автоматически будет использовать CUDA, если она доступна на вашем компьютере, 
# в противном случае она будет использовать центральный процессор. 
# Вы можете применить устройство по вашему выбору, передав его имя в качестве аргумента конструктору. 
# Загрузка модели с помощью CUDA может занять несколько секунд, 
# но затем она выполняется очень быстро.
encoder = VoiceEncoder()

In [6]:
# Мы будем использовать уменьшенную версию набора данных LibriSpeech test-other для запуска наших примеров. 
# Этот меньший набор данных содержит 10 динамиков с 10 произнесениями каждый. 
# Примечание. "wav" в именах переменных означает "форма волны", а не расширение файла wav.
# wav_fpaths = list(Path("audio_data", "librispeech_test-other").glob("**/*.flac"))

# В данном случае собственные использованы файлы с записанной речью 
wav_fpaths = list(Path('Voices').glob('**/*.wav'))

In [ ]:
# Сгруппируем волны для каждого спикера и загрузим их, используя функцию предварительной обработки, 
# предоставляемую similablyzer, для загрузки сигналов в память. 
# Он нормализует громкость, сокращает длительные паузы 
# и выполняет повторную выборку до правильной частоты дискретизации.
                            # map(функция обработки сигнала, массив wav_fpaths для применения функции)     
speaker_wavs = {speaker: list(map(preprocess_wav, wav_fpaths)) for speaker, wav_fpaths in
                groupby(tqdm(wav_fpaths, "Preprocessing wavs", len(wav_fpaths), unit="wavs"), 
                        lambda wav_fpath: wav_fpath.parent.stem)}

In [ ]:
# сравниваем в рамках папок одного спикера (комбинации своих голосов)

from itertools import combinations
import time

start_time = time.time()

num_true=0
num_total=0

# ради сравнения перебираем значения словаря (т.е. эмбединги записей спикеров) и создаем список всех возможных комбинаций:
# (спикер 1 запись 1 - спикер 1 запись 2), (спикер 1 запись 1 - спикер 1 запись 3) и т.д.
for elems in speaker_wavs.values():   
    tuples = list(combinations(elems, 2)) # получаем перебор всех комбинаций
  
    for single in tuples:         # идем по каждому кортежу в списке
        # функция .embed_utterance() создает эмбединги голосов внутри каждой папки
        embeds = (np.array( [encoder.embed_utterance(single[0]) ] ), np.array([encoder.embed_utterance(single[1]) ] ) )
        num_total+=1

        # Вычислим матрицу подобия. Сходство двух эмбедингов - это просто их точечное произведение, 
        # потому что метрикой сходства является косинусное сходство, а эмбединги уже нормированы по L2.
        # Короткая версия:
        utt_sim_matrix = np.inner(embeds[0], embeds[1])    # Внутреннее произведение двух массивов. 
        if utt_sim_matrix > 0.67:
            num_true+=1
        # Длинная, подробная версия (это исполнять необязательно):
        utt_sim_matrix2 = np.zeros( (len(embeds[0]), len(embeds[1]) ) )
        for i in range(len(embeds[0])):
            for j in range(len(embeds[1])):
                # Обозначение @ эквивалентно np.dot(embedds_a[i], embedds_b[i])
                utt_sim_matrix2[i, j] = embeds[0][i] @ embeds[1][j]
                # np.allclose() возвращает True, если два массива равны по элементам в пределах допуска
        assert np.allclose(utt_sim_matrix, utt_sim_matrix2)
                    

# считаем метрику accuracy:
if num_total !=0:
    accuracy = num_true/num_total
    print('Точность: ', accuracy)
    print('Количество файлов: ', num_total)
else:
    print('На ноль делить нельзя')

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# подход распознавания на основе esemblyzer: сравниваем несвои голоса 
from itertools import product, chain
from os import walk
import time

start_time = time.time()

nums_true=0
nums_total=0
 
for wavs in speaker_wavs.copy():
    elems = speaker_wavs[wavs]     # wavs - аналог root, а elems - аналог files
    del speaker_wavs[wavs] 
    
    # собираем по одному файлу для каждого списка (в итоге получаем идентичные 12 списков)
    another_files = list(chain(*[speaker_wavs[key] for key in speaker_wavs]))
 
    # Пример: product('ABCD', 'xy') --> Ax Ay Bx By Cx Cy Dx Dy
    results = list(product(elems, another_files))
    for pairs in results:
        embeddings = (np.array( [encoder.embed_utterance(pairs[0]) ] ), np.array([encoder.embed_utterance(pairs[1]) ] ) )
        nums_total+=1
        speech_sim_matrix = np.inner(embeddings[0], embeddings[1])    
        if speech_sim_matrix < 0.67:
            num_true+=1
     
 
 #  считаем метрику accuracy:
if nums_total !=0:
    accuracy = nums_true/nums_total
    print('Accuracy ', accuracy)
    print('Количество файлов: ', nums_total)
else:
    print('На ноль делить нельзя')

print("--- %s seconds ---" % (time.time() - start_time))